### Ollama sdk  
  
Source: [Ollama Docs](https://docs.ollama.com/capabilities/tool-calling#python-2)

In [2]:
pip install -r requirements.txt

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.0 MB 4.6 MB/s eta 0:00:01
   ------------------------------- -------- 1.6/2.0 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 4.4 MB/s  0:00:00

   -------------------- -------------------  5/10 [pydantic]
   -------------------- -------------------  5/10 [pydantic]
   ---------------------------- -----------  7/10 [anyio]
   ---------------------------------------- 10/10 [ollama]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from ollama import chat

In [19]:
# Define the message to send to the model
message = """
        How to implement fibbonaci sequence in Python?
        Provide code only.
        """
print(message)


        How to implement fibbonaci sequence in Python?
        Provide code only.
        


### Thinking

In [20]:
# Define the chat interaction with the model
response = chat(
    model='gpt-oss:20b',
    messages=[{'role': 'user',
               'content':message}],
               think=True,
               stream=False
)

print('Thinking: \n', response.message.thinking, '\n')
print('Answer: \n', response.message.content)

Thinking: 
 The user wants to implement Fibonacci sequence in Python, provide code only. So just code. Probably define function fib, maybe using recursion or iterative. Provide code only, no explanation. They didn't specify any constraints, but likely a simple implementation. I'll produce code that defines function fib(n) returns nth Fibonacci number. And maybe also a generator for sequence. Provide only code. 

Answer: 
 ```python
def fibonacci(n: int) -> int:
    """Return the nth Fibonacci number (0-indexed)."""
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a

# Example usage:
# print([fibonacci(i) for i in range(10)])
```


In [16]:
print(response.message.content)

```python
def quicksort(arr):
    def _quicksort(a, low, high):
        if low < high:
            pivot_index = partition(a, low, high)
            _quicksort(a, low, pivot_index - 1)
            _quicksort(a, pivot_index + 1, high)

    def partition(a, low, high):
        pivot = a[high]
        i = low - 1
        for j in range(low, high):
            if a[j] <= pivot:
                i += 1
                a[i], a[j] = a[j], a[i]
        a[i + 1], a[high] = a[high], a[i + 1]
        return i + 1

    _quicksort(arr, 0, len(arr) - 1)
    return arr

# Example usage:
# sorted_list = quicksort([3, 6, 8, 10, 1, 2, 1])
```


### Tools calling

In [32]:
# Define tool 
def square_nums(number: int) -> int:
    """This function returns the square of a number.
    Args:
        number (int): The number to be squared."""
    results = number * number
    
    return results

square_nums(23)

529

In [37]:
# Define the message to send to the model and use the tool
message = """
        What is the square of 23?
"""
messages = [
    {'role': 'user',
     'content': message,
    }
]

# pass tool to the model
response = chat(
    model='gpt-oss:20b',
    messages=messages,
    tools=[square_nums],
    think=True)

print('Thinking: \n', response.message.thinking, '\n')

Thinking: 
 We need to use the function square_nums. Should output result. Probably we must call the function via 'functions.square_nums'. 



In [34]:
print('Answer: \n', response.message.content)

Answer: 
 


In [35]:
# ...
messages.append(response.message)
if response.message.tool_calls:
  # only recommended for models which only return a single tool call
  call = response.message.tool_calls[0]
  result = square_nums(**call.function.arguments)
  # add the tool result to the messages
  messages.append({"role": "tool", 
                   "tool_name": call.function.name, 
                   "content": str(result)})

  final_response = chat(model="gpt-oss:20b", 
                        messages=messages, 
                        tools=[square_nums], 
                        think=True)
  print('Final Thinking: \n', final_response.message.thinking, '\n')


Final Thinking: 
 None 



In [36]:
print(final_response.message.content)

The square of 23 is 529.
